In [36]:
from datetime import datetime
from collections import Counter

In [1]:
mongo = gmcdb('production', 'jaipur')

2024-02-12T14:47:25.665078Z [info     ] Mongos instances selected      environment=research_sb1 file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/connection_string_builder.py function_name=get_selected_mongos_instances hostname=Dima-Shulga-MacBook-Pro hosts=['applicativedb-prod-mongos-2.omcomcom.com', 'applicativedb-prod-mongos-0.omcomcom.com'] line_number=38 module=hs_gimme.db_facade.connection_string_builder pid=12035


In [3]:
notifications = list(mongo.notification_events.find({'_created_at': {'$gt': datetime(2023, 11, 1)}}))
len(notifications)

2024-02-12T14:49:31.135009Z [warning  ] SlowLog                        command_name=saslContinue connection_id=applicativedb-prod-mongos-0.omcomcom.com:27017 duration_ms__numeric=11707.338 duration_threshold__numeric=10000 environment=research_sb1 file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga-MacBook-Pro line_number=11 module=hs_gimme.db_facade.slow_logs_utils pid=12035 request_id=1954899097 source=PyMongo stack_snippet=
  File "/Users/dima/.asdf/installs/python/3.9.5/lib/python3.9/contextlib.py", line 117, in __enter__
    return next(self.gen)

2024-02-12T14:50:00.948250Z [warning  ] SlowLog                        command_name=find connection_id=applicativedb-prod-mongos-0.omcomcom.com:27017 duration_ms__numeric=29806.042 duration_threshold__numeric=10000 environment=research_sb1 file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga

19969

In [12]:
sorted(notifications[0]['payload'].keys())

['account_id',
 'bcc_address',
 'entity_id',
 'entity_type',
 'environment',
 'initiating_user_id',
 'notification_type',
 'preset',
 'preset_context',
 'recommendation_context',
 'recommended_req_ids',
 'signature',
 'source_system',
 'talent_id',
 'task_name']

In [25]:
data = []
for notif in notifications:
    d = dict(notif)
    for k, v in notif['payload'].items():
        d['payload__' + k] = v
    data.append(d)
df = pd.DataFrame(data)

In [26]:
df.columns

Index(['_id', '_created_at', '_updated_at', 'message_history',
       'notification_type', 'custom_id', 'status', 'rpa_audit_id',
       'notification_action', 'recipient', 'service_provider', 'payload',
       'payload__initiating_user_id', 'payload__account_id',
       'payload__talent_id', 'payload__recommended_req_ids',
       'payload__preset_context', 'payload__recommendation_context',
       'payload__preset', 'payload__notification_type', 'payload__entity_type',
       'payload__entity_id', 'payload__source_system', 'payload__signature',
       'payload__environment', 'payload__task_name', 'payload__bcc_address'],
      dtype='object')

In [31]:
df['payload__task_name'].value_counts(dropna=False)

get_open_for_opportunities_program            7067
NaN                                           5655
hm_call_for_action_v2                         4532
recruiter_sla_digest                          2522
get_internal_applications_for_redeployment     115
auto_fetch_on_new_req                           78
Name: payload__task_name, dtype: int64

In [39]:

for f in ['opened', 'clicked_on_url', 'blocked', 'unsubscribed', 'spam']:
    df['is_' + f] = df['message_history'].apply(lambda x: f in {v['event_subtype'] for v in x})
    print('is_' + f, df['is_' + f].mean())

is_opened 0.4434373278581802
is_clicked_on_url 0.5932695678301367
is_blocked 0.05753918573789373
is_unsubscribed 0.0019029495718363464
is_spam 0.00045069858280334516


In [42]:
df.groupby(['notification_action', 'payload__task_name'], dropna=False).agg(is_opened=('is_opened', 'mean'), 
                                     is_clicked_on_url=('is_clicked_on_url', 'mean'), 
                                     is_blocked=('is_blocked', 'mean'), 
                                     is_unsubscribed=('is_unsubscribed', 'mean'), 
                                     is_spam=('is_spam', 'mean'))

,,is_opened,is_clicked_on_url,is_blocked,is_unsubscribed,is_spam
notification_action,payload__task_name,,,,,
auto_fetch_on_new_req,auto_fetch_on_new_req,0.461538,0.115385,0.423077,0.012821,0.000000
get_internal_applications_for_redeployment,get_internal_applications_for_redeployment,0.478261,0.652174,0.156522,0.000000,0.000000
get_open_for_opportunities_program,get_open_for_opportunities_program,0.327862,0.871232,0.081647,0.002264,0.000000
hm_call_for_action_v2,hm_call_for_action_v2,0.380847,0.646072,0.094219,0.000000,0.000000
recruiter_sla_digest,recruiter_sla_digest,0.372720,0.726011,0.017843,0.000793,0.000000
sending_fetch_mail,NaN,0.668612,0.149779,0.008665,0.003360,0.001592
